<a href="https://colab.research.google.com/github/yashpandey030303/SLM-Quantization-research-implementation/blob/main/SLM_Quantisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("--- Step 1: Forcefully uninstalling old libraries ---")
!pip uninstall -y transformers accelerate

--- Step 1: Forcefully uninstalling old libraries ---
Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1


In [ ]:
print("\n--- Step 1.1: Installing the correct, latest libraries ---")
!pip install -q "accelerate>=0.28.0"
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets bitsandbytes


--- Step 1.1: Installing the correct, latest libraries ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.17.1 requires transformers, which is not installed.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.6 MB/s eta 0:00:00


In [ ]:
!pip show transformers

Name: transformers
Version: 4.57.0.dev0
Summary: Transformers: the model-definition framework for state-of-the-art machine learning models in text, vision, audio, and multimodal models, for both inference and training.
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
import torch
import time
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset

print("Libraries installed and imported successfully!\n")


Libraries installed and imported successfully!



In [ ]:
print("--- Step 2: Configuring models ---")
model_ids = [
    "microsoft/Phi-3-mini-4k-instruct",
    "Qwen/Qwen1.5-1.8B-Chat"
]
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
print("Model IDs and quantization config are set.\n")

--- Step 2: Configuring models ---
Model IDs and quantization config are set.



In [ ]:
print("--- Step 3: Loading and preparing the XNLI dataset ---")

# Load the first 50 samples from the English and Hindi validation sets directly.
try:
    en_samples = load_dataset("xnli", "en", split="validation").select(range(50))
    hi_samples = load_dataset("xnli", "hi", split="validation").select(range(50))
    xnli_samples = {"en": en_samples, "hi": hi_samples}

    # The labels are numeric (0=entailment, 1=neutral, 2=contradiction). We map them to text for the model.
    label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}

    print(f"Loaded {len(en_samples)} English and {len(hi_samples)} Hindi samples.\n")

except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")
    print("Please ensure you have a stable internet connection.")

--- Step 3: Loading and preparing the XNLI dataset ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

en/train-00000-of-00001.parquet:   0%|          | 0.00/50.2M [00:00<?, ?B/s]

en/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

hi/validation-00000-of-00001.parquet:   0%|          | 0.00/249k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Loaded 50 English and 50 Hindi samples.



In [ ]:
def evaluate_model(model_id, quantization_config=None):
    """
    Loads a model and evaluates its performance on the XNLI samples.
    Measures accuracy, average latency, and peak GPU memory usage.
    """
    print(f"\nLoading tokenizer for {model_id}...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Set device_map: Quantized models MUST be on GPU. Baseline can be auto (may offload to CPU).
    device_map_setting = "cuda" if quantization_config else "auto"
    print(f"Loading model {model_id} with device_map='{device_map_setting}' and {'4-bit quantization' if quantization_config else 'no quantization'}...")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=quantization_config,
            device_map=device_map_setting,
            trust_remote_code=True,
            attn_implementation="sdpa", # Use SDPA for potentially better memory/speed and stability
            torch_dtype=torch.bfloat16 # Use bfloat16 for Phi-3 baseline if not quantized, for better memory
        )
    except Exception as e:
        print(f"ERROR: Failed to load model {model_id}. This is often due to OOM or incompatibilities.")
        print(f"Details: {e}")
        return [] # Return empty list to prevent further errors for this model

    results = []

    print(f"\n--- Evaluating Model: {model_id} ({'4-bit Quantized' if quantization_config else 'Baseline'}) ---")

    for lang, samples in xnli_samples.items():
        correct_predictions = 0
        total_latency = 0

        # Reset GPU memory stats for accurate measurement per language
        # This will only be relevant if the model itself fits on GPU
        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

        for i, example in enumerate(samples):
            premise = example['premise']
            hypothesis = example['hypothesis']
            true_label = label_map[example['label']]

            prompt = (
                f"Premise: '{premise}'\n"
                f"Hypothesis: '{hypothesis}'\n"
                "Based on the premise, does the hypothesis mean entailment, neutral, or contradiction? "
                "Answer with only one word."
            )

            messages = [{"role": "user", "content": prompt}]
            inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

            # Add attention_mask to avoid warning and for more reliable results
            # And use_cache=False to avoid DynamicCache related errors, as discussed
            start_time = time.time()
            outputs = model.generate(
                inputs,
                max_new_tokens=5,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask=inputs.attention_mask if hasattr(inputs, 'attention_mask') else None,
                use_cache=False # Crucial for stability with potentially problematic caches, and for small outputs like 5 tokens, performance hit is minimal.
            )
            end_time = time.time()
            total_latency += (end_time - start_time)

            response_text = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).lower().strip()

            if true_label in response_text:
                correct_predictions += 1

            # Print progress
            if (i + 1) % 10 == 0 or (i + 1) == len(samples): # Print every 10 samples or at the end
                print(f"  [{lang.upper()}] Sample {i+1}/{len(samples)} | Pred: '{response_text}' | True: '{true_label}'")

        # Calculate metrics
        accuracy = (correct_predictions / len(samples)) * 100
        avg_latency = total_latency / len(samples)
        peak_memory_mb = torch.cuda.max_memory_allocated() / (1024 * 1024) if torch.cuda.is_available() else 0 # Handle CPU offload

        results.append({
            "Model": model_id.split('/')[-1],
            "Type": "4-bit Quantized" if quantization_config else "Baseline",
            "Language": lang.upper(),
            "Accuracy (%)": f"{accuracy:.2f}",
            "Avg Latency (s/ex)": f"{avg_latency:.4f}",
            "Peak VRAM (MB)": f"{peak_memory_mb:.2f}"
        })

    # Clear memory after evaluation for this model
    del model
    del tokenizer
    torch.cuda.empty_cache()

    return results

print("Evaluation function is ready.\n")

Evaluation function is ready.



In [ ]:
print("--- Step 5: Starting experiments incrementally ---")
results_filename = "slm_quantization_results.csv"

# Clear previous results file if it exists
if os.path.exists(results_filename):
    os.remove(results_filename)
    print(f"Removed existing '{results_filename}'.")

# Header for the CSV file
csv_header = ["Model", "Type", "Language", "Accuracy (%)", "Avg Latency (s/ex)", "Peak VRAM (MB)"]

--- Step 5: Starting experiments incrementally ---


NameError: name 'os' is not defined

In [ ]:
print("\n--- Running Experiment 1: Phi-3 Baseline ---")
model_id_phi = "microsoft/Phi-3-mini-4k-instruct"
phi_baseline_results = evaluate_model(model_id_phi, quantization_config=None)
df_phi_baseline = pd.DataFrame(phi_baseline_results, columns=csv_header)
df_phi_baseline.to_csv(results_filename, mode='w', header=True, index=False)
print(f"Phi-3 Baseline results saved to {results_filename}")


--- Running Experiment 1: Phi-3 Baseline ---

Loading tokenizer for microsoft/Phi-3-mini-4k-instruct...


The module name microsoft/Phi_hyphen_3_hyphen_mini_hyphen_4k_hyphen_instruct (originally microsoft/Phi-3-mini-4k-instruct) is not a valid Python identifier. Please rename the original module to avoid import issues.
The module name microsoft/Phi_hyphen_3_hyphen_mini_hyphen_4k_hyphen_instruct (originally microsoft/Phi-3-mini-4k-instruct) is not a valid Python identifier. Please rename the original module to avoid import issues.


Loading model microsoft/Phi-3-mini-4k-instruct with device_map='auto' and no quantization...
ERROR: Failed to load model microsoft/Phi-3-mini-4k-instruct. This is often due to OOM or incompatibilities.
Details: Phi3ForCausalLM does not support an attention implementation through torch.nn.functional.scaled_dot_product_attention yet. Please request the support for this architecture: https://github.com/huggingface/transformers/issues/28005. If you believe this error is a bug, please open an issue in Transformers GitHub repository and load your model with the argument `attn_implementation="eager"` meanwhile. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="eager")`


NameError: name 'csv_header' is not defined

In [ ]:
print("\n--- Running Experiment 1: Phi-3 Baseline ---")
model_id_phi = "microsoft/Phi-3-mini-4k-instruct"
phi_baseline_results = evaluate_model(model_id_phi, quantization_config=None)
df_phi_baseline = pd.DataFrame(phi_baseline_results, columns=csv_header)
df_phi_baseline.to_csv(results_filename, mode='w', header=True, index=False)
print(f"✅ Phi-3 Baseline results saved to {results_filename}")

In [ ]:
# --- Experiment 3: Qwen Baseline ---
print("\n--- Running Experiment 3: Qwen Baseline ---")
model_id_qwen = "Qwen/Qwen1.5-1.8B-Chat"
qwen_baseline_results = evaluate_model(model_id_qwen, quantization_config=None)
df_qwen_baseline = pd.DataFrame(qwen_baseline_results, columns=csv_header)
df_qwen_baseline.to_csv(results_filename, mode='a', header=False, index=False)
print(f"✅ Qwen Baseline results appended to {results_filename}")

In [ ]:
# --- Experiment 4: Qwen Quantized ---
print("\n--- Running Experiment 4: Qwen Quantized ---")
model_id_qwen = "Qwen/Qwen1.5-1.8B-Chat"
qwen_quantized_results = evaluate_model(model_id_qwen, quantization_config=quantization_config)
df_qwen_quantized = pd.DataFrame(qwen_quantized_results, columns=csv_header)
df_qwen_quantized.to_csv(results_filename, mode='a', header=False, index=False)
print(f"✅ Qwen Quantized results appended to {results_filename}"

In [ ]:
print("\n--- Final Results Table ---")

try:
    final_results_df = pd.read_csv(results_filename)
    print(final_results_df.to_string())
except FileNotFoundError:
    print("The results file was not found. This should not happen if all experiments ran correctly.")